In [0]:
-- setting to allow easy querying of tables in analytics
SET search_path TO analytics;

In [0]:
-- fact item pricing has duplicates issue, hence resolving here using distinct
CREATE TEMP TABLE item_pricing AS
SELECT DISTINCT price_date, item_id, price_2
FROM fact_item_pricing;

-- inserting missing dates in the date lookup table
INSERT INTO adhoc.date_list_for_dr
(
    SELECT DISTINCT wrt_cng_dat AS dt FROM users_views.written_sales_transactions AS t
    LEFT JOIN adhoc.date_list_for_dr AS l ON t.wrt_cng_dat = l.dt
    WHERE dt IS NULL
);
-- inserting if there are still missing dates
INSERT INTO adhoc.date_list_for_dr
(
    SELECT (start_date + n) :: date AS dt
    FROM (SELECT MAX(dt) AS start_date, CURRENT_DATE - 1 AS end_date FROM adhoc.date_list_for_dr),
        (SELECT ROW_NUMBER() OVER() AS n FROM dim_pc)       -- use any small table
    WHERE start_date + n <= end_date
);

In [0]:
-- the new view for wrt transactions doesn't have app_key, manually managing it here
-- also removing unknown profit center '0'
DROP TABLE IF EXISTS fact_trxn_temp;
CREATE TEMP TABLE fact_trxn_temp AS
WITH txn_cte AS (
    SELECT *, CASE WHEN LOWER(location) = 'greensboro' THEN 120
        WHEN LOWER(location) = 'mobile' THEN 116 ELSE 91 END :: bigint AS app_key
    FROM users_views.written_sales_transactions WHERE wrt_pft_ctr <> '0'
),

all_combos AS (
    SELECT dt AS wrt_cng_dat, app_key, wrt_pft_ctr FROM adhoc.date_list_for_dr AS r
    CROSS JOIN (SELECT DISTINCT app_key, wrt_pft_ctr FROM txn_cte)
)

-- adding dummy rows for dates/stores that are missing to help with wtd/mtd/ytd calculations
SELECT wrt_cng_dat, app_key, wrt_pft_ctr, NVL(wrt_cat, 'n/a') AS wrt_cat, NVL(wrt_cust_id, 'n/a') AS wrt_cust_id,
    NVL(wrt_item_id, 'n/a') AS wrt_item_id, NVL(wrt_vend_id, 'n/a') AS wrt_vend_id, NVL(wrt_terms, 'n/a') AS wrt_terms,
    NVL(wrt_so_no, '0000') AS wrt_so_no, NVL(wrt_so_dtl_seq, '00') AS wrt_so_dtl_seq, NVL(wrt_tax, 'n/a') AS wrt_tax,
    NVL(wrt_cogs, 0) AS wrt_cogs, NVL(wrt_sls, 0) AS wrt_sls, NVL(wrt_qty_sld, 0) AS wrt_qty_sld,
    NVL(wrt_dist_amt_1, 0) AS wrt_dist_amt_1, NVL(wrt_dist_amt_2, 0) AS wrt_dist_amt_2,
    NVL(wrt_dist_amt_3, 0) AS wrt_dist_amt_3, NVL(wrt_dist_amt_4, 0) AS wrt_dist_amt_4,
    NVL(wrt_dist_amt_5, 0) AS wrt_dist_amt_5, NVL(wrt_dist_amt_6, 0) AS wrt_dist_amt_6,
    NVL(wrt_dist_amt_7, 0) AS wrt_dist_amt_7, NVL(wrt_dist_amt_8, 0) AS wrt_dist_amt_8
FROM all_combos AS c LEFT JOIN txn_cte AS t USING (wrt_cng_dat, app_key, wrt_pft_ctr);

In [0]:
DROP TABLE IF EXISTS wrt_deposit_temp_table;
CREATE TEMP TABLE wrt_deposit_temp_table AS
-- getting pc data ready for further calculations
WITH cte_pc AS (
    SELECT app_key, pc_key,

        (STRPOS(dist_code_1, 'FIN') = 0) :: smallint AS check_1,

        (STRPOS(dist_code_2, 'FIN') = 0) :: smallint AS check_2,

        (STRPOS(dist_code_3, 'FIN') = 0) :: smallint AS check_3,

        (STRPOS(dist_code_4, 'FIN') = 0) :: smallint AS check_4,

        (STRPOS(dist_code_5, 'FIN') = 0) :: smallint AS check_5,

        (STRPOS(dist_code_6, 'FIN') = 0) :: smallint AS check_6,

        (STRPOS(dist_code_7, 'FIN') = 0) :: smallint AS check_7,

        (STRPOS(dist_code_8, 'FIN') = 0) :: smallint AS check_8

    FROM dim_pc
),

-- summing the dist columns to get the deposit amount
dep_cte1 AS (
    SELECT wrt_cng_dat AS report_date, t.app_key, wrt_pft_ctr, wrt_so_no, wrt_terms,
        wrt_dist_amt_1 * check_1 + wrt_dist_amt_2 * check_2 + wrt_dist_amt_3 * check_3 +
        wrt_dist_amt_4 * check_4 + wrt_dist_amt_5 * check_5 + wrt_dist_amt_6 * check_6 +
        wrt_dist_amt_7 * check_7 + wrt_dist_amt_8 * check_8 AS deposit_amount
    FROM fact_trxn_temp AS t
        LEFT JOIN cte_pc AS c ON t.wrt_pft_ctr = c.pc_key AND t.app_key = c.app_key
    WHERE wrt_cat NOT IN ('ZY', 'ZZ')
),

-- aggregating to get the abs value, sign count
dep_cte2 AS (
    SELECT report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms,
        MIN(ABS(deposit_amount)) AS abs_deposit, SUM(CASE WHEN deposit_amount = 0 THEN 0
        ELSE deposit_amount / ABS(deposit_amount) END) AS sign_sum
    FROM dep_cte1 GROUP BY 1, 2, 3, 4, 5
)

-- finalizing the deposit amount
SELECT report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms,
    abs_deposit * (CASE WHEN sign_sum = 0 THEN 0 WHEN sign_sum > 0
    THEN 1 ELSE -1 END) AS deposit_amount
FROM dep_cte2;

In [0]:
DROP TABLE IF EXISTS wrt_tax_temp_table;
CREATE TEMP TABLE wrt_tax_temp_table AS
-- getting tax data ready for further calculations
with cte_tax AS (
    SELECT f.*,

        CASE WHEN wrt_sls = 0 THEN 0
        WHEN tax_pct_1_max = 0 THEN tax_pct_1 * wrt_sls / 100
        WHEN (tax_pct_1 * wrt_sls / 100) > tax_pct_1_max THEN tax_pct_1_max
        WHEN (tax_pct_1 * wrt_sls / 100) < -tax_pct_1_max THEN -tax_pct_1_max
        ELSE tax_pct_1 * wrt_sls / 100 END AS tax_1,

        CASE WHEN wrt_sls = 0 THEN 0
        WHEN tax_pct_2_max = 0 THEN tax_pct_2 * wrt_sls / 100
        WHEN (tax_pct_2 * wrt_sls / 100) > tax_pct_2_max THEN tax_pct_2_max
        WHEN (tax_pct_2 * wrt_sls / 100) < -tax_pct_2_max THEN -tax_pct_2_max
        ELSE tax_pct_2 * wrt_sls / 100 END AS tax_2,

        CASE WHEN wrt_sls = 0 THEN 0
        WHEN tax_pct_3_max = 0 THEN tax_pct_3 * wrt_sls / 100
        WHEN (tax_pct_3 * wrt_sls / 100) > tax_pct_3_max THEN tax_pct_3_max
        WHEN (tax_pct_3 * wrt_sls / 100) < -tax_pct_3_max THEN -tax_pct_3_max
        ELSE tax_pct_3 * wrt_sls / 100 END AS tax_3,

        ROUND(NVL(current_fee, 0) / 100, 4) AS current_fee_percent

    FROM fact_trxn_temp AS f
        LEFT JOIN dim_tax AS t ON f.wrt_tax = t.tax_code AND f.app_key = t.app_key
        LEFT JOIN terms_fee_lookup AS l ON TRIM(f.wrt_terms) = UPPER(TRIM(l.term_code))
),

-- adding sales and tax
cte1 AS (
    SELECT wrt_cng_dat AS report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms, wrt_cat, wrt_vend_id,
        wrt_item_id, wrt_cogs, wrt_sls, tax_1 + tax_2 + tax_3 AS tax_sum, current_fee_percent
    FROM cte_tax AS t
),

cte2 AS (
    SELECT report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms, SUM(wrt_sls) AS sales_sum,
        SUM(tax_sum) AS tax_sum, MIN(current_fee_percent) AS finance_fee_percent
    FROM cte1 AS t
    WHERE wrt_cat NOT IN ('ZY', 'ZZ')
    GROUP BY 1, 2, 3, 4, 5
),

-- metric: bedding sales, cogs
cte3 AS (
    SELECT report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms, SUM(wrt_sls) AS bedding_sales_sum,
        SUM(wrt_cogs) AS bedding_cogs_sum, SUM(tax_sum) AS bedding_tax_sum
    FROM cte1
    WHERE wrt_vend_id IN ('PURPLE', 'TEMPUR', 'SEALY', 'BEDGEAR', 'SIMMONS', 'SERTA') OR
        wrt_item_id IN ('*GUARDIAN10A', '*GUARDIAN5A', '*GDN10A/SPKG', '*GDN5A/SPKG') OR
        wrt_cat IN ('20A', '20B', '20D', '20P', '20S', 'AS', 'BB', 'BF', 'BI', 'BL', 'BV', 'CF', 'CI', 'CV', 'S20')
    GROUP BY 1, 2, 3, 4, 5
),

-- metric: furniture sales and cogs
cte4 AS (
    SELECT report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms, SUM(wrt_sls) AS furniture_sales_sum,
        SUM(wrt_cogs) AS furniture_cogs_sum, SUM(tax_sum) AS furniture_tax_sum
    FROM cte1
    WHERE wrt_vend_id NOT IN ('PURPLE', 'TEMPUR', 'SEALY', 'BEDGEAR', 'SIMMONS', 'SERTA', 'GUARDIANPROD', 'GUARDIAN') AND
        wrt_item_id NOT IN ('*GUARDIAN10A', '*GUARDIAN5A', '*GDN10A/SPKG', '*GDN5A/SPKG') AND
        wrt_cat NOT IN ('20A', '20B', '20D', '20P', '20S', 'AS', 'BB', 'BF', 'BI', 'BL', 'BV', 'CF', 'CI', 'CV', 'S20', 'ZDE', 'ZDS')
    GROUP BY 1, 2, 3, 4, 5
),

-- so number level financing calculation
cte5 AS (
    SELECT report_date, app_key, wrt_pft_ctr, wrt_so_no,
        SUM(tax_sum) AS tax_amount, SUM(deposit_amount) AS deposit_amount,
        SUM(sales_sum + tax_sum - deposit_amount) AS financed_amount,
        SUM((sales_sum + tax_sum - deposit_amount) * finance_fee_percent) AS financing_cost,

        SUM(bedding_sales_sum) AS bedding_sales_sum, SUM(bedding_cogs_sum) AS bedding_cogs_sum,
        SUM(furniture_sales_sum) AS furniture_sales_sum, SUM(furniture_cogs_sum) AS furniture_cogs_sum,
        
        SUM(bedding_sales_sum + bedding_tax_sum) * SUM((sales_sum + tax_sum - deposit_amount)
        * finance_fee_percent) / NULLIF(SUM(sales_sum + tax_sum), 0) AS bedding_financing_cost,

        SUM(furniture_sales_sum + furniture_tax_sum) * SUM((sales_sum + tax_sum - deposit_amount)
        * finance_fee_percent) / NULLIF(SUM(sales_sum + tax_sum), 0) AS furniture_financing_cost

    FROM cte2
    JOIN wrt_deposit_temp_table USING (report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms)
    FULL OUTER JOIN cte3 USING (report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms)
    FULL OUTER JOIN cte4 USING (report_date, app_key, wrt_pft_ctr, wrt_so_no, wrt_terms)
    GROUP BY 1, 2, 3, 4
)

SELECT * FROM cte5;

In [0]:
DROP TABLE IF EXISTS report_part_1;
CREATE TEMP TABLE report_part_1 AS
-- metrics: finance metrics, bedding & furniture
WITH fin_cte AS (
    SELECT report_date, app_key, wrt_pft_ctr,
        ROUND(SUM(tax_amount), 2) AS total_tax_amount,
        ROUND(SUM(deposit_amount), 2) AS total_deposit_amount,
        ROUND(SUM(financed_amount), 2) AS total_financed_amount,
        ROUND(SUM(financing_cost), 2) AS total_financing_cost,
        ROUND(SUM(bedding_financing_cost), 2) AS bedding_financing_cost,
        ROUND(SUM(furniture_financing_cost), 2) AS furniture_financing_cost,
        ROUND(SUM(bedding_sales_sum), 2) AS bedding_sales,
        ROUND(SUM(bedding_cogs_sum), 2) AS bedding_cogs,
        ROUND(SUM(furniture_sales_sum), 2) AS furniture_sales,
        ROUND(SUM(furniture_cogs_sum), 2) AS furniture_cogs
    FROM wrt_tax_temp_table AS t
    GROUP BY 1, 2, 3
),

-- metrics: customers financed, sales financed
cte1_base AS (
    SELECT DISTINCT f.wrt_cng_dat, f.app_key, f.wrt_pft_ctr, wrt_cust_id
    FROM fact_trxn_temp AS f JOIN wrt_tax_temp_table AS t
        ON f.wrt_cng_dat = t.report_date AND f.app_key = t.app_key
        AND f.wrt_pft_ctr = t.wrt_pft_ctr AND f.wrt_so_no = t.wrt_so_no
    WHERE financed_amount :: int > 0
),
cte1 AS (
    SELECT f.wrt_cng_dat AS report_date, f.app_key, f.wrt_pft_ctr, SUM(wrt_sls) AS sales_financed,
        COUNT(DISTINCT f.wrt_cust_id) AS customers_financed
    FROM fact_trxn_temp AS f JOIN cte1_base AS b
        USING (wrt_cng_dat, app_key, wrt_pft_ctr, wrt_cust_id)
    WHERE wrt_cat NOT IN ('ZY', 'ZZ')
    GROUP BY 1, 2, 3
),

-- metrics: total sales, total cogs and total promo - excluding categories zy, zz
cte2 AS (
    SELECT wrt_cng_dat AS report_date, f.app_key, wrt_pft_ctr,
        SUM(wrt_sls) AS total_sales, SUM(wrt_cogs) AS total_cogs
        -- , SUM(NVL(wrt_qty_sld * price_2, wrt_sls)) AS total_promo_price
    FROM fact_trxn_temp AS f
    LEFT JOIN item_pricing AS i ON f.wrt_cng_dat = i.price_date
        AND LTRIM(f.wrt_item_id, '0') = LTRIM(i.item_id, '0')       -- may need to modify later
    WHERE wrt_cat NOT IN ('ZY', 'ZZ')
    GROUP BY 1, 2, 3
),

-- metric: protection sales
cte3 AS (
    SELECT wrt_cng_dat AS report_date, app_key, wrt_pft_ctr, SUM(wrt_sls) AS protection_sales
    FROM fact_trxn_temp
    WHERE wrt_vend_id IN ('GUARDIANPROD', 'GUARDIAN')
    GROUP BY 1, 2, 3
),

-- pre-aggregating paid hours for each location and joining multiple tables to get the needed columns
-- here nvl for date is a dummy value, we just want it to be beyond 1 year date value
-- (need to change when all dates are provided)
payroll_cte AS (
    SELECT work_date, location_code, SUM(paid_hours) AS paid_hours
    FROM payroll GROUP BY 1, 2
),
cte4 AS (
    SELECT f.*, store_name, region, visual_proof_opps :: int AS traffic, paid_hours
    FROM fin_cte AS f
    LEFT JOIN store_lookup AS l ON f.wrt_pft_ctr :: int = l.pft_ctr
        AND f.app_key = l.app_key
    LEFT JOIN store_traffic AS st ON f.report_date = st.traffic_date :: date
        AND l.etrax_shopping_id = st.showroom_id
    LEFT JOIN payroll_cte AS p ON f.report_date = p.work_date
        AND l.paylocity_location_id = p.location_code
),

-- finally joining all ctes to get all the metrics in one place
final AS (
    SELECT * FROM cte1
    FULL OUTER JOIN cte2 USING (report_date, app_key, wrt_pft_ctr)
    FULL OUTER JOIN cte3 USING (report_date, app_key, wrt_pft_ctr)
    FULL OUTER JOIN cte4 USING (report_date, app_key, wrt_pft_ctr)
)

SELECT * FROM final;

In [0]:
DROP TABLE IF EXISTS report_part_2;
CREATE TEMP TABLE report_part_2 AS
-- preparing a multi-level sales, quantity, cogs, promo price sums
WITH wrt_pivot AS (
    SELECT wrt_cng_dat AS report_date, t.app_key, wrt_pft_ctr, wrt_cust_id, wrt_so_no,
        wrt_cat, wrt_so_dtl_seq, SUM(wrt_sls) AS sales_sum, SUM(wrt_qty_sld) AS qty_sum,
        SUM(wrt_cogs) AS cogs_sum, SUM(NVL(wrt_qty_sld * price_2, wrt_sls)) AS promo_price_sum
    FROM fact_trxn_temp AS t
    LEFT JOIN item_pricing AS i ON t.wrt_cng_dat = i.price_date
        AND LTRIM(t.wrt_item_id, '0') = LTRIM(i.item_id, '0')       -- may need to modify later
    GROUP BY 1, 2, 3, 4, 5, 6, 7
),

-- metric: cancelled sales - i.e negative sales
cte1 AS (
    SELECT report_date, app_key, wrt_pft_ctr, SUM(sales_sum) AS cancelled_sales
    FROM wrt_pivot
    WHERE sales_sum < 0 AND wrt_cat NOT IN ('ZY', 'ZZ')
    GROUP BY 1, 2, 3
),

-- metrics: number of unique customers, number of unique customers with discount
cte2 AS (
    SELECT report_date, app_key, wrt_pft_ctr, COUNT(DISTINCT wrt_cust_id) AS unique_cust_count,
        COUNT(DISTINCT CASE WHEN sales_sum < promo_price_sum THEN wrt_cust_id END) AS discounted_cust_count,
        SUM(sales_sum) AS all_sales_exc_ctgs, SUM(cogs_sum) AS all_cogs_exc_ctgs,
        SUM(promo_price_sum) AS all_promo_exc_ctgs
    FROM wrt_pivot
    WHERE sales_sum > 0 AND wrt_cat NOT IN ('ZDE', 'ZDS', 'ZFA', 'ZY', 'ZZ')
    GROUP BY 1, 2, 3
),

-- metric: delivery sales - only for categories zde, zds
cte3 AS (
    SELECT report_date, app_key, wrt_pft_ctr, SUM(sales_sum) AS delivery_sales
    FROM wrt_pivot
    WHERE wrt_cat IN ('ZDE', 'ZDS')
    GROUP BY 1, 2, 3
),

-- metrics: excluding zde, zds, zy, zz
-- calculate unique customers count, total sales, total quantities
cte4_base AS (
    SELECT report_date, app_key, wrt_pft_ctr, wrt_cust_id, wrt_so_no,
        SUM(sales_sum) AS sales_sum_exc_ctgs, SUM(qty_sum) AS qty_sum_exc_ctgs
    FROM wrt_pivot
    WHERE wrt_cat NOT IN ('ZDE', 'ZDS', 'ZFA', 'ZY', 'ZZ')
    GROUP BY 1, 2, 3, 4, 5 HAVING sales_sum_exc_ctgs >= 100
),
cte4 AS (
    SELECT report_date, app_key, wrt_pft_ctr, COUNT(DISTINCT wrt_cust_id) AS cust_count_exc_ctgs,
        SUM(sales_sum_exc_ctgs) AS sales_exc_ctgs, SUM(qty_sum_exc_ctgs) AS qty_exc_ctgs
    FROM cte4_base
    GROUP BY 1, 2, 3
),

-- finally joining all ctes to get all the metrics in one place
final AS (
    SELECT * FROM cte1
    FULL OUTER JOIN cte2 USING (report_date, app_key, wrt_pft_ctr)
    FULL OUTER JOIN cte3 USING (report_date, app_key, wrt_pft_ctr)
    FULL OUTER JOIN cte4 USING (report_date, app_key, wrt_pft_ctr)
)

SELECT * FROM final;

In [0]:
DROP TABLE IF EXISTS sales_his_part;
CREATE TEMP TABLE sales_his_part AS
WITH sales_his_cte AS (
    SELECT purchase_date AS report_date, CASE WHEN LOWER(location_key) = 'greensboro'
        THEN 120 WHEN LOWER(location_key) = 'mobile' THEN 116 ELSE 91 END :: bigint
        AS app_key, profit_center AS wrt_pft_ctr, item_id, category, vendor_id, sales_amount,
        cogs, quantity_sold, DATEDIFF(d, sales_order_date, purchase_date) AS days_to_deliver
    FROM fact_sales_history
),

-- metrics: sales, cogs, quantity delivered & weighted days
cte1 AS (
    SELECT report_date, sh.app_key, wrt_pft_ctr, store_name AS store_name_sh,
        region AS region_sh, SUM(sales_amount) AS sales_delivered, SUM(cogs) AS cogs_delivered,
        SUM(quantity_sold) AS qty_delivered, SUM(quantity_sold * days_to_deliver) AS weighted_days
    FROM sales_his_cte AS sh
    LEFT JOIN store_lookup AS l ON sh.wrt_pft_ctr :: int = l.pft_ctr
        AND sh.app_key = l.app_key
    -- WHERE category NOT IN ('ZY', 'ZZ')
    GROUP BY 1, 2, 3, 4, 5
),

-- metric: bedding sales delivered
cte2 AS (
    SELECT report_date, app_key, wrt_pft_ctr, SUM(sales_amount) AS bedding_sales_delivered
    FROM sales_his_cte
    WHERE vendor_id IN ('PURPLE', 'TEMPUR', 'SEALY', 'BEDGEAR', 'SIMMONS', 'SERTA') OR
        item_id IN ('*GUARDIAN10A', '*GUARDIAN5A', '*GDN10A/SPKG', '*GDN5A/SPKG') OR
        category IN ('20A', '20B', '20D', '20P', '20S', 'AS', 'BB', 'BF', 'BI', 'BL', 'BV', 'CF', 'CI', 'CV', 'S20')
    GROUP BY 1, 2, 3
),

-- metric: carts count for carts built
cte3 AS (
    SELECT eventdate AS report_date, CASE WHEN LOWER(location) = 'greensboro' THEN 120
        WHEN LOWER(location) = 'mobile' THEN 116 ELSE 91 END :: bigint AS app_key,
        profitcenter AS wrt_pft_ctr, COUNT(*) AS carts_count
    FROM stage_ah_cart_report WHERE guestname IS NOT NULL AND guestpersonemail IS NOT NULL
        AND guestphone IS NOT NULL AND LOWER(stagename) = 'saved shopping cart'
    GROUP BY 1, 2, 3
),

-- metric: finance applications count
cte4 AS (
    SELECT eventdate AS report_date, sl.app_key, sl.pft_ctr :: varchar AS wrt_pft_ctr,
        COUNT(DISTINCT customer_email) AS applications_count
    FROM stage_lendpro AS lp
    LEFT JOIN store_lookup AS sl ON lp.storeid :: int = sl.lendpro_store_id
    GROUP BY 1, 2, 3
)

SELECT * FROM cte1
FULL OUTER JOIN cte2 USING (report_date, app_key, wrt_pft_ctr)
FULL OUTER JOIN cte3 USING (report_date, app_key, wrt_pft_ctr)
FULL OUTER JOIN cte4 USING (report_date, app_key, wrt_pft_ctr)
;

In [0]:
DROP TABLE IF EXISTS report_part_3;
CREATE TEMP TABLE report_part_3 AS
-- combining report part 1 and 2; creating additional columns needed for ease of power bi report generation
WITH cte1 AS (
SELECT report_date, app_key, wrt_pft_ctr AS profit_center,
    report_date || ';' || app_key || ';' || wrt_pft_ctr AS custom_key,
    'Individual' AS filter_level, NVL(store_name, store_name_sh, 'unknown') AS segment,
    NVL(store_name, store_name_sh, 'unknown') AS store_name,
    NVL(region, region_sh, 'n/a') AS region,
    NVL(traffic, 0) AS traffic,
    NVL(total_sales, 0) AS total_sales_amt,
    NVL(CASE WHEN wrt_pft_ctr NOT IN ('444', '445') THEN total_sales END, 0) AS stores_sales_amt,
    NVL(CASE WHEN wrt_pft_ctr = '444' THEN total_sales END, 0) AS online_sales_amt,
    NVL(CASE WHEN wrt_pft_ctr = '445' THEN total_sales END, 0) AS omni_sales_amt,
    NVL(total_cogs, 0) AS total_cogs,
    NVL(CASE WHEN wrt_pft_ctr NOT IN ('444', '445') THEN total_cogs END, 0) AS total_cogs_stores,
    NVL(total_tax_amount, 0) AS total_tax_amt,
    NVL(total_deposit_amount, 0) AS total_deposit_amt,
    NVL(carts_count, 0) AS carts_count,
    NVL(applications_count, 0) AS applications_count,
    NVL(paid_hours, 0) AS paid_hours,
    NVL(bedding_sales, 0) AS bedding_sales_amt,
    NVL(protection_sales, 0) AS protection_sales_amt,
    NVL(furniture_sales, 0) AS furniture_sales_amt,
    NVL(furniture_cogs, 0) AS furniture_cogs,
    NVL(bedding_cogs, 0) AS bedding_cogs,
    NVL(furniture_financing_cost, 0) AS furniture_financing_cost,
    NVL(bedding_financing_cost, 0) AS bedding_financing_cost,
    NVL(delivery_sales, 0) AS delivery_sales_amt,
    NVL(cancelled_sales, 0) AS cancelled_sales_amt,
    NVL(sales_financed, 0) AS sales_financed,
    NVL(customers_financed, 0) AS customers_financed,
    NVL(total_financed_amount, 0) AS total_financed_amt,
    NVL(total_financing_cost, 0) AS total_financing_cost,
    NVL(unique_cust_count, 0) AS unique_cust_count,
    NVL(discounted_cust_count, 0) AS discounted_cust_count,
    NVL(all_sales_exc_ctgs, 0) AS all_sales_exc_ctgs,
    NVL(all_cogs_exc_ctgs, 0) AS all_cogs_exc_ctgs,
    NVL(all_promo_exc_ctgs, 0) AS all_promo_exc_ctgs,
    NVL(cust_count_exc_ctgs, 0) AS cust_count_exc_ctgs,
    NVL(sales_exc_ctgs, 0) AS sales_exc_ctgs,
    NVL(qty_exc_ctgs, 0) AS qty_exc_ctgs,
    NVL(sales_delivered, 0) AS sales_delivered,
    NVL(cogs_delivered, 0) AS cogs_delivered,
    NVL(qty_delivered, 0) AS qty_delivered,
    NVL(weighted_days, 0) AS weighted_days,
    NVL(bedding_sales_delivered, 0) AS bedding_sales_delivered
FROM report_part_1
FULL OUTER JOIN report_part_2 USING (report_date, app_key, wrt_pft_ctr)
FULL OUTER JOIN sales_his_part USING (report_date, app_key, wrt_pft_ctr)),

cte2 AS (
SELECT DISTINCT report_date, DATE_PART(w, report_date) || ' - '
    || DATE_PART(dw, report_date) AS week_day_num
FROM cte1),

cte3 AS (
SELECT *, NTH_VALUE(report_date, 2) OVER (PARTITION BY week_day_num
    ORDER BY report_date DESC ROWS BETWEEN UNBOUNDED PRECEDING AND
    UNBOUNDED FOLLOWING) AS ly_date
FROM cte2)

SELECT ly_date, a.* FROM cte1 AS a JOIN cte3 USING (report_date);

In [0]:
-- stored procedure for calulating numerator/denominator type metrics
-- p_target is the target temp table name, p_from is the source temp table name
CREATE OR REPLACE PROCEDURE sp_dr_num_den (p_target VARCHAR, p_from VARCHAR)
LANGUAGE plpgsql
AS $$
DECLARE
    sql_stmt VARCHAR(MAX);
BEGIN
    sql_stmt := '
        CREATE TEMP TABLE ' || p_target || ' AS
        SELECT *,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND((total_sales_amt - total_cogs) * 100
            / total_sales_amt, 2), 2) END AS gross_margin_pct,

            TRUNC(ROUND(stores_sales_amt - total_cogs_stores, 2), 2) AS stores_gross_margin_amt,

            CASE WHEN stores_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND((stores_sales_amt
            - total_cogs_stores) * 100 / stores_sales_amt, 2), 2) END AS stores_gross_margin_pct,

            CASE WHEN all_promo_exc_ctgs = 0 THEN 0 ELSE TRUNC(ROUND((all_promo_exc_ctgs -
            all_cogs_exc_ctgs) * 100 / all_promo_exc_ctgs, 2), 2) END AS initial_markup_pct,

            CASE WHEN all_promo_exc_ctgs = 0 THEN 0 ELSE TRUNC(ROUND((all_promo_exc_ctgs -
            all_sales_exc_ctgs) * 100 / all_promo_exc_ctgs, 2), 2) END AS item_discount_pct,

            CASE WHEN unique_cust_count = 0 THEN 0 ELSE TRUNC(ROUND(discounted_cust_count * 100.00
            / unique_cust_count, 2), 2) END AS orders_with_discount_pct,

            TRUNC(ROUND(total_sales_amt - total_cogs - total_financing_cost, 2), 2) AS effective_margin_amt,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND((total_sales_amt - total_cogs
            - total_financing_cost) * 100 / total_sales_amt, 2), 2) END AS effective_margin_pct,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND(total_financing_cost * 100
            / total_sales_amt, 2), 2) END AS financing_cost_pct,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(total_sales_amt
            / traffic, 2), 2) END AS sales_per_guest,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(cust_count_exc_ctgs * 100.00
            / traffic, 2), 2) END AS close_rate_pct,

            CASE WHEN cust_count_exc_ctgs = 0 THEN 0 ELSE TRUNC(ROUND(sales_exc_ctgs
            / cust_count_exc_ctgs, 2), 2) END AS avg_ticket_amt,

            CASE WHEN cust_count_exc_ctgs = 0 THEN 0 ELSE TRUNC(ROUND(qty_exc_ctgs * 1.00
            / cust_count_exc_ctgs, 2), 2) END AS items_per_ticket,

            CASE WHEN qty_exc_ctgs = 0 THEN 0 ELSE TRUNC(ROUND(sales_exc_ctgs
            / qty_exc_ctgs, 2), 2) END AS avg_sales_price,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(carts_count * 100.00
            / traffic, 2), 2) END AS carts_built_pct,

            CASE WHEN paid_hours = 0 THEN 0 ELSE TRUNC(ROUND(total_sales_amt
            / paid_hours, 2), 2) END AS sales_per_hour,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND(total_financed_amt * 100
            / total_sales_amt, 2), 2) END AS financed_sales_pct,

            CASE WHEN customers_financed = 0 THEN 0 ELSE TRUNC(ROUND(sales_financed
            / customers_financed, 2), 2) END AS avg_financed_ticket_amt,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(applications_count * 100.00
            / traffic, 2), 2) END AS finance_application_pct,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(furniture_sales_amt
            / traffic, 2), 2) END AS furniture_spg,

            CASE WHEN paid_hours = 0 THEN 0 ELSE TRUNC(ROUND(furniture_sales_amt
            / paid_hours, 2), 2) END AS furniture_sph,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND(furniture_sales_amt * 100
            / total_sales_amt, 2), 2) END AS furniture_sales_pct,

            CASE WHEN furniture_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND((furniture_sales_amt 
            - furniture_cogs) * 100 / furniture_sales_amt, 2), 2) END AS furniture_gross_margin_pct,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(bedding_sales_amt
            / traffic, 2), 2) END AS bedding_spg,

            CASE WHEN paid_hours = 0 THEN 0 ELSE TRUNC(ROUND(bedding_sales_amt
            / paid_hours, 2), 2) END AS bedding_sph,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND(bedding_sales_amt * 100
            / total_sales_amt, 2), 2) END AS bedding_sales_pct,

            CASE WHEN bedding_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND((bedding_sales_amt 
            - bedding_cogs) * 100 / bedding_sales_amt, 2), 2) END AS bedding_gross_margin_pct,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(protection_sales_amt
            / traffic, 2), 2) END AS protection_spg,

            CASE WHEN paid_hours = 0 THEN 0 ELSE TRUNC(ROUND(protection_sales_amt
            / paid_hours, 2), 2) END AS protection_sph,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND(protection_sales_amt * 100
            / total_sales_amt, 2), 2) END AS protection_sales_pct,

            CASE WHEN traffic = 0 THEN 0 ELSE TRUNC(ROUND(delivery_sales_amt
            / traffic, 2), 2) END AS delivery_spg,

            CASE WHEN total_sales_amt = 0 THEN 0 ELSE TRUNC(ROUND(delivery_sales_amt * 100
            / total_sales_amt, 2), 2) END AS delivery_sales_pct,

            CASE WHEN sales_delivered = 0 THEN 0 ELSE TRUNC(ROUND((sales_delivered
            - cogs_delivered) * 100 / sales_delivered, 2), 2) END AS delivered_gross_margin_pct,

            CASE WHEN qty_delivered = 0 THEN 0 ELSE TRUNC(ROUND(weighted_days
            / qty_delivered, 2), 2) END AS avg_days_to_deliver

        FROM ' || p_from;
    EXECUTE sql_stmt;
    RAISE INFO 'Temp table % has been created.', p_target;
END;
$$;

In [0]:
-- calculating remaining daily metrics and saving as report_part_4 temp table
-- parameters to be passed as strings
CALL sp_dr_num_den('report_part_4', 'report_part_3');

In [0]:
-- stored procedure for calulating aggregate level metrics
-- p_target is the target temp table name, p_from is the source temp table name
-- p_type is daily/wtd/mtd/ytd, agg_type is total/in-stores/online/omni/region
CREATE OR REPLACE PROCEDURE sp_dr_agg_by
    (p_target VARCHAR, p_from VARCHAR, agg_type VARCHAR, p_type VARCHAR, p_is_comp BOOL)
LANGUAGE plpgsql
AS $$
DECLARE
    start_part VARCHAR;
    region_part VARCHAR;
    comp_part1 VARCHAR(100);
    comp_part2 VARCHAR;
    sum_part VARCHAR(MAX);
    where_part VARCHAR;
    group_part VARCHAR;
    ord_part VARCHAR(1000);
    sql_stmt VARCHAR(MAX);
BEGIN
    CASE p_type
    WHEN 'daily' THEN start_part := '';
    WHEN 'week' THEN start_part := 'week_start, ';
    WHEN 'month' THEN start_part := 'month_start, ';
    WHEN 'year' THEN start_part := 'year_start, ';
    ELSE RAISE EXCEPTION 'Wrong input for p_type: %', p_type;
    END CASE;

    IF agg_type <> 'region' THEN 
        region_part := '';
    ELSE
        region_part := 'region, ';
    END IF;

    IF p_is_comp THEN
        comp_part1 := 'CASE WHEN ly_date >= store_open_date THEN ';
        comp_part2 := ' END';
    ELSE
        comp_part1 := ''; comp_part2 := '';
    END IF;

    sum_part := 'SUM(' || comp_part1 || 'traffic' || comp_part2 || ') AS traffic,
        SUM(' || comp_part1 || 'total_sales_amt' || comp_part2 || ') AS total_sales_amt,
        SUM(' || comp_part1 || 'stores_sales_amt' || comp_part2 || ') AS stores_sales_amt,
        SUM(' || comp_part1 || 'online_sales_amt' || comp_part2 || ') AS online_sales_amt,
        SUM(' || comp_part1 || 'omni_sales_amt' || comp_part2 || ') AS omni_sales_amt,
        SUM(' || comp_part1 || 'total_cogs' || comp_part2 || ') AS total_cogs,
        SUM(' || comp_part1 || 'total_cogs_stores' || comp_part2 || ') AS total_cogs_stores,
        SUM(' || comp_part1 || 'total_tax_amt' || comp_part2 || ') AS total_tax_amt,
        SUM(' || comp_part1 || 'total_deposit_amt' || comp_part2 || ') AS total_deposit_amt,
        SUM(' || comp_part1 || 'carts_count' || comp_part2 || ') AS carts_count,
        SUM(' || comp_part1 || 'applications_count' || comp_part2 || ') AS applications_count,
        SUM(' || comp_part1 || 'paid_hours' || comp_part2 || ') AS paid_hours,
        SUM(' || comp_part1 || 'bedding_sales_amt' || comp_part2 || ') AS bedding_sales_amt,
        SUM(' || comp_part1 || 'protection_sales_amt' || comp_part2 || ') AS protection_sales_amt,
        SUM(' || comp_part1 || 'furniture_sales_amt' || comp_part2 || ') AS furniture_sales_amt,
        SUM(' || comp_part1 || 'furniture_cogs' || comp_part2 || ') AS furniture_cogs,
        SUM(' || comp_part1 || 'bedding_cogs' || comp_part2 || ') AS bedding_cogs,
        SUM(' || comp_part1 || 'furniture_financing_cost' || comp_part2 || ') AS furniture_financing_cost,
        SUM(' || comp_part1 || 'bedding_financing_cost' || comp_part2 || ') AS bedding_financing_cost,
        SUM(' || comp_part1 || 'delivery_sales_amt' || comp_part2 || ') AS delivery_sales_amt,
        SUM(' || comp_part1 || 'cancelled_sales_amt' || comp_part2 || ') AS cancelled_sales_amt,
        SUM(' || comp_part1 || 'sales_financed' || comp_part2 || ') AS sales_financed,
        SUM(' || comp_part1 || 'customers_financed' || comp_part2 || ') AS customers_financed,
        SUM(' || comp_part1 || 'total_financed_amt' || comp_part2 || ') AS total_financed_amt,
        SUM(' || comp_part1 || 'total_financing_cost' || comp_part2 || ') AS total_financing_cost,
        SUM(' || comp_part1 || 'unique_cust_count' || comp_part2 || ') AS unique_cust_count,
        SUM(' || comp_part1 || 'discounted_cust_count' || comp_part2 || ') AS discounted_cust_count,
        SUM(' || comp_part1 || 'all_sales_exc_ctgs' || comp_part2 || ') AS all_sales_exc_ctgs,
        SUM(' || comp_part1 || 'all_cogs_exc_ctgs' || comp_part2 || ') AS all_cogs_exc_ctgs,
        SUM(' || comp_part1 || 'all_promo_exc_ctgs' || comp_part2 || ') AS all_promo_exc_ctgs,
        SUM(' || comp_part1 || 'cust_count_exc_ctgs' || comp_part2 || ') AS cust_count_exc_ctgs,
        SUM(' || comp_part1 || 'sales_exc_ctgs' || comp_part2 || ') AS sales_exc_ctgs,
        SUM(' || comp_part1 || 'qty_exc_ctgs' || comp_part2 || ') AS qty_exc_ctgs,
        SUM(' || comp_part1 || 'sales_delivered' || comp_part2 || ') AS sales_delivered,
        SUM(' || comp_part1 || 'cogs_delivered' || comp_part2 || ') AS cogs_delivered,
        SUM(' || comp_part1 || 'qty_delivered' || comp_part2 || ') AS qty_delivered,
        SUM(' || comp_part1 || 'weighted_days' || comp_part2 || ') AS weighted_days,
        SUM(' || comp_part1 || 'bedding_sales_delivered' || comp_part2 || ') AS bedding_sales_delivered';

    CASE
    WHEN p_type = 'daily' AND agg_type <> 'region' THEN group_part := '';
    WHEN p_type = 'daily' OR agg_type <> 'region' THEN group_part := ', 3';
    ELSE group_part := ', 3, 4';
    END CASE;

    CASE agg_type
    WHEN 'total' THEN
        where_part := '';
        ord_part := 'report_date || '';total'' AS custom_key, ''Aggregated'' AS filter_level,
                     ''Total'' AS segment, ''n/a'' AS store_name, ''n/a'' AS region,';
    WHEN 'in-stores' THEN
        where_part := ' WHERE profit_center NOT IN (''444'', ''445'')';
        ord_part := 'report_date || '';in-stores'' AS custom_key, ''Aggregated'' AS filter_level,
                     ''In-Stores'' AS segment, ''n/a'' AS store_name, ''n/a'' AS region,';
    WHEN 'online' THEN
        where_part := ' WHERE profit_center = ''444''';
        ord_part := 'report_date || '';online'' AS custom_key, ''Aggregated'' AS filter_level,
                     ''Online'' AS segment, ''n/a'' AS store_name, ''n/a'' AS region,';
    WHEN 'omni' THEN
        where_part := ' WHERE profit_center = ''445''';
        ord_part := 'report_date || '';omni'' AS custom_key, ''Aggregated'' AS filter_level,
                     ''Omni'' AS segment, ''n/a'' AS store_name, ''n/a'' AS region,';
    ELSE
        where_part := ' WHERE LOWER(region) IN (''east'', ''west'')';
        ord_part := 'report_date || '';'' || region AS custom_key, ''Aggregated'' AS filter_level,
                     region AS segment, ''n/a'' AS store_name, region,';
    END CASE;

    sql_stmt := '
        CREATE TEMP TABLE ' || p_target || ' AS
        WITH agg_cte AS (
        SELECT ' || start_part || 'ly_date, report_date, ' || region_part || sum_part || '
        FROM ' || p_from || where_part || ' GROUP BY 1, 2' || group_part || ')

        SELECT ' || start_part || 'ly_date, report_date, NULL :: bigint AS app_key, ''n/a'' AS profit_center, '
            || ord_part || ' traffic, total_sales_amt, stores_sales_amt, online_sales_amt, omni_sales_amt,
            total_cogs, total_cogs_stores, total_tax_amt, total_deposit_amt, carts_count, applications_count,
            paid_hours, bedding_sales_amt, protection_sales_amt, furniture_sales_amt, furniture_cogs,
            bedding_cogs, furniture_financing_cost, bedding_financing_cost, delivery_sales_amt,
            cancelled_sales_amt, sales_financed, customers_financed, total_financed_amt, total_financing_cost,
            unique_cust_count, discounted_cust_count, all_sales_exc_ctgs, all_cogs_exc_ctgs,
            all_promo_exc_ctgs, cust_count_exc_ctgs, sales_exc_ctgs, qty_exc_ctgs, sales_delivered,
            cogs_delivered, qty_delivered, weighted_days, bedding_sales_delivered
        FROM agg_cte;';
    EXECUTE sql_stmt;
    RAISE INFO 'Full statement: %', sql_stmt;
    RAISE INFO 'Temp table % has been created.', p_target;
END;
$$;

In [0]:
CALL sp_dr_agg_by('report_part_5_base', 'report_part_3', 'total', 'daily', FALSE);

CALL sp_dr_num_den('report_part_5', 'report_part_5_base');

DROP TABLE report_part_5_base;

In [0]:
CALL sp_dr_agg_by('report_part_6_base', 'report_part_3', 'in-stores', 'daily', FALSE);

CALL sp_dr_num_den('report_part_6', 'report_part_6_base');

DROP TABLE report_part_6_base;

In [0]:
CALL sp_dr_agg_by('report_part_7_base', 'report_part_3', 'online', 'daily', FALSE);

CALL sp_dr_num_den('report_part_7', 'report_part_7_base');

DROP TABLE report_part_7_base;

In [0]:
CALL sp_dr_agg_by('report_part_8_base', 'report_part_3', 'omni', 'daily', FALSE);

CALL sp_dr_num_den('report_part_8', 'report_part_8_base');

DROP TABLE report_part_8_base;

In [0]:
CALL sp_dr_agg_by('report_part_9_base', 'report_part_3', 'region', 'daily', FALSE);

CALL sp_dr_num_den('report_part_9', 'report_part_9_base');

DROP TABLE report_part_9_base;

In [0]:
-- creating the daily report table by unioning all the report parts
DROP TABLE IF EXISTS daily_report CASCADE;
CREATE TABLE daily_report AS
SELECT * FROM report_part_4 UNION
SELECT * FROM report_part_5 UNION
SELECT * FROM report_part_6 UNION
SELECT * FROM report_part_7 UNION
SELECT * FROM report_part_8 UNION
SELECT * FROM report_part_9;

DROP TABLE report_part_4; DROP TABLE report_part_5;
DROP TABLE report_part_6; DROP TABLE report_part_7;
DROP TABLE report_part_8; DROP TABLE report_part_9;

In [0]:
-- % LY calculations
DROP TABLE IF EXISTS ly_daily_report CASCADE;
CREATE TABLE ly_daily_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM daily_report AS f LEFT JOIN daily_report AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

In [0]:
-- the following is a placeholder date (will be taken off once all the open dates are available)
CREATE TEMP TABLE comp_prep AS
SELECT NVL(grand_open_date, '2020-01-01') :: date AS store_open_date, r.*
FROM report_part_3 AS r LEFT JOIN store_lookup AS l
ON r.profit_center :: int = l.pft_ctr AND r.app_key = l.app_key;

In [0]:
CALL sp_dr_agg_by('report_comp_1_base', 'comp_prep', 'total', 'daily', TRUE);

CALL sp_dr_num_den('report_comp_1', 'report_comp_1_base');

DROP TABLE report_comp_1_base;

In [0]:
CALL sp_dr_agg_by('report_comp_2_base', 'comp_prep', 'in-stores', 'daily', TRUE);

CALL sp_dr_num_den('report_comp_2', 'report_comp_2_base');

DROP TABLE report_comp_2_base;

In [0]:
CALL sp_dr_agg_by('report_comp_3_base', 'comp_prep', 'online', 'daily', TRUE);

CALL sp_dr_num_den('report_comp_3', 'report_comp_3_base');

DROP TABLE report_comp_3_base;

In [0]:
CALL sp_dr_agg_by('report_comp_4_base', 'comp_prep', 'omni', 'daily', TRUE);

CALL sp_dr_num_den('report_comp_4', 'report_comp_4_base');

DROP TABLE report_comp_4_base;

In [0]:
CALL sp_dr_agg_by('report_comp_5_base', 'comp_prep', 'region', 'daily', TRUE);

CALL sp_dr_num_den('report_comp_5', 'report_comp_5_base');

DROP TABLE report_comp_5_base;

In [0]:
CREATE TEMP TABLE comp_daily AS
SELECT * FROM report_comp_1 UNION
SELECT * FROM report_comp_2 UNION
SELECT * FROM report_comp_3 UNION
SELECT * FROM report_comp_4 UNION
SELECT * FROM report_comp_5;

DROP TABLE report_comp_1; DROP TABLE report_comp_2;
DROP TABLE report_comp_3; DROP TABLE report_comp_4;
DROP TABLE report_comp_5; DROP TABLE comp_prep;

In [0]:
-- % LY comp calculations
DROP TABLE IF EXISTS comp_daily_report CASCADE;
CREATE TABLE comp_daily_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM comp_daily AS f LEFT JOIN comp_daily AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

DROP TABLE comp_daily;

In [0]:
-- stored procedure for calulating week/month/year to date metrics
-- p_target is the target temp table name, p_from is the source temp table name
-- p_type is week/month/year
CREATE OR REPLACE PROCEDURE sp_dr_to_date (p_type VARCHAR, p_target VARCHAR, p_from VARCHAR)
LANGUAGE plpgsql
AS $$
DECLARE
    date_trunc_expr VARCHAR;
    window_clause VARCHAR(MAX);
    sql_stmt VARCHAR(MAX);
BEGIN
    date_trunc_expr := 'DATE_TRUNC(''' || p_type || ''', report_date)';

    window_clause := '(PARTITION BY ' || date_trunc_expr || ', store_name ORDER BY report_date
                        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)';

    EXECUTE 'DROP TABLE IF EXISTS ' || p_target;

    sql_stmt := '
        CREATE TEMP TABLE ' || p_target || ' AS
        SELECT ' || date_trunc_expr || ' :: DATE AS ' || p_type || '_start, ly_date, report_date,
            app_key, profit_center, custom_key, filter_level, segment, store_name, region,
            SUM(traffic) OVER ' || window_clause || ' AS traffic,
            SUM(total_sales_amt) OVER ' || window_clause || ' AS total_sales_amt,
            SUM(stores_sales_amt) OVER ' || window_clause || ' AS stores_sales_amt,
            SUM(online_sales_amt) OVER ' || window_clause || ' AS online_sales_amt,
            SUM(omni_sales_amt) OVER ' || window_clause || ' AS omni_sales_amt,
            SUM(total_cogs) OVER ' || window_clause || ' AS total_cogs,
            SUM(total_cogs_stores) OVER ' || window_clause || ' AS total_cogs_stores,
            SUM(total_tax_amt) OVER ' || window_clause || ' AS total_tax_amt,
            SUM(total_deposit_amt) OVER ' || window_clause || ' AS total_deposit_amt,
            SUM(carts_count) OVER ' || window_clause || ' AS carts_count,
            SUM(applications_count) OVER ' || window_clause || ' AS applications_count,
            SUM(paid_hours) OVER ' || window_clause || ' AS paid_hours,
            SUM(bedding_sales_amt) OVER ' || window_clause || ' AS bedding_sales_amt,
            SUM(protection_sales_amt) OVER ' || window_clause || ' AS protection_sales_amt,
            SUM(furniture_sales_amt) OVER ' || window_clause || ' AS furniture_sales_amt,
            SUM(furniture_cogs) OVER ' || window_clause || ' AS furniture_cogs,
            SUM(bedding_cogs) OVER ' || window_clause || ' AS bedding_cogs,
            SUM(furniture_financing_cost) OVER ' || window_clause || ' AS furniture_financing_cost,
            SUM(bedding_financing_cost) OVER ' || window_clause || ' AS bedding_financing_cost,
            SUM(delivery_sales_amt) OVER ' || window_clause || ' AS delivery_sales_amt,
            SUM(cancelled_sales_amt) OVER ' || window_clause || ' AS cancelled_sales_amt,
            SUM(sales_financed) OVER ' || window_clause || ' AS sales_financed,
            SUM(customers_financed) OVER ' || window_clause || ' AS customers_financed,
            SUM(total_financed_amt) OVER ' || window_clause || ' AS total_financed_amt,
            SUM(total_financing_cost) OVER ' || window_clause || ' AS total_financing_cost,
            SUM(unique_cust_count) OVER ' || window_clause || ' AS unique_cust_count,
            SUM(discounted_cust_count) OVER ' || window_clause || ' AS discounted_cust_count,
            SUM(all_sales_exc_ctgs) OVER ' || window_clause || ' AS all_sales_exc_ctgs,
            SUM(all_cogs_exc_ctgs) OVER ' || window_clause || ' AS all_cogs_exc_ctgs,
            SUM(all_promo_exc_ctgs) OVER ' || window_clause || ' AS all_promo_exc_ctgs,
            SUM(cust_count_exc_ctgs) OVER ' || window_clause || ' AS cust_count_exc_ctgs,
            SUM(sales_exc_ctgs) OVER ' || window_clause || ' AS sales_exc_ctgs,
            SUM(qty_exc_ctgs) OVER ' || window_clause || ' AS qty_exc_ctgs,
            SUM(sales_delivered) OVER ' || window_clause || ' AS sales_delivered,
            SUM(cogs_delivered) OVER ' || window_clause || ' AS cogs_delivered,
            SUM(qty_delivered) OVER ' || window_clause || ' AS qty_delivered,
            SUM(weighted_days) OVER ' || window_clause || ' AS weighted_days,
            SUM(bedding_sales_delivered) OVER ' || window_clause || ' AS bedding_sales_delivered
        FROM ' || p_from;
    RAISE INFO 'Temp table % has been created.', p_target;
    EXECUTE sql_stmt;
END;
$$;

In [0]:
-- calculating week to date metrics
CALL sp_dr_to_date('week', 'wtd_part1', 'report_part_3');

-- calculating rest of the week to date metrics
CALL sp_dr_num_den('wtd_part2', 'wtd_part1');

In [0]:
CALL sp_dr_agg_by('wtd_part3_base', 'wtd_part1', 'total', 'week', FALSE);

CALL sp_dr_num_den('wtd_part3', 'wtd_part3_base');

DROP TABLE wtd_part3_base;

In [0]:
CALL sp_dr_agg_by('wtd_part4_base', 'wtd_part1', 'in-stores', 'week', FALSE);

CALL sp_dr_num_den('wtd_part4', 'wtd_part4_base');

DROP TABLE wtd_part4_base;

In [0]:
CALL sp_dr_agg_by('wtd_part5_base', 'wtd_part1', 'online', 'week', FALSE);

CALL sp_dr_num_den('wtd_part5', 'wtd_part5_base');

DROP TABLE wtd_part5_base;

In [0]:
CALL sp_dr_agg_by('wtd_part6_base', 'wtd_part1', 'omni', 'week', FALSE);

CALL sp_dr_num_den('wtd_part6', 'wtd_part6_base');

DROP TABLE wtd_part6_base;

In [0]:
CALL sp_dr_agg_by('wtd_part7_base', 'wtd_part1', 'region', 'week', FALSE);

CALL sp_dr_num_den('wtd_part7', 'wtd_part7_base');

DROP TABLE wtd_part7_base;

In [0]:
-- creating the wtd report table by unioning all the wtd parts
DROP TABLE IF EXISTS wtd_report CASCADE;
CREATE TABLE wtd_report AS
SELECT * FROM wtd_part2 UNION
SELECT * FROM wtd_part3 UNION
SELECT * FROM wtd_part4 UNION
SELECT * FROM wtd_part5 UNION
SELECT * FROM wtd_part6 UNION
SELECT * FROM wtd_part7;

DROP TABLE wtd_part2; DROP TABLE wtd_part3;
DROP TABLE wtd_part4; DROP TABLE wtd_part5;
DROP TABLE wtd_part6; DROP TABLE wtd_part7;

In [0]:
-- % LY calculations
DROP TABLE IF EXISTS ly_wtd_report CASCADE;
CREATE TABLE ly_wtd_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM wtd_report AS f LEFT JOIN wtd_report AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

In [0]:
CREATE TEMP TABLE wtd_comp_prep AS
SELECT NVL(grand_open_date, '2020-01-01') :: date AS store_open_date, r.*
FROM wtd_part1 AS r LEFT JOIN store_lookup AS l
ON r.profit_center :: int = l.pft_ctr AND r.app_key = l.app_key;

In [0]:
CALL sp_dr_agg_by('wtd_comp_1_base', 'wtd_comp_prep', 'total', 'week', TRUE);

CALL sp_dr_num_den('wtd_comp_1', 'wtd_comp_1_base');

DROP TABLE wtd_comp_1_base;

In [0]:
CALL sp_dr_agg_by('wtd_comp_2_base', 'wtd_comp_prep', 'in-stores', 'week', TRUE);

CALL sp_dr_num_den('wtd_comp_2', 'wtd_comp_2_base');

DROP TABLE wtd_comp_2_base;

In [0]:
CALL sp_dr_agg_by('wtd_comp_3_base', 'wtd_comp_prep', 'online', 'week', TRUE);

CALL sp_dr_num_den('wtd_comp_3', 'wtd_comp_3_base');

DROP TABLE wtd_comp_3_base;

In [0]:
CALL sp_dr_agg_by('wtd_comp_4_base', 'wtd_comp_prep', 'omni', 'week', TRUE);

CALL sp_dr_num_den('wtd_comp_4', 'wtd_comp_4_base');

DROP TABLE wtd_comp_4_base;

In [0]:
CALL sp_dr_agg_by('wtd_comp_5_base', 'wtd_comp_prep', 'region', 'week', TRUE);

CALL sp_dr_num_den('wtd_comp_5', 'wtd_comp_5_base');

DROP TABLE wtd_comp_5_base;

In [0]:
CREATE TEMP TABLE comp_wtd AS
SELECT * FROM wtd_comp_1 UNION
SELECT * FROM wtd_comp_2 UNION
SELECT * FROM wtd_comp_3 UNION
SELECT * FROM wtd_comp_4 UNION
SELECT * FROM wtd_comp_5;

DROP TABLE wtd_comp_1; DROP TABLE wtd_comp_2;
DROP TABLE wtd_comp_3; DROP TABLE wtd_comp_4;
DROP TABLE wtd_comp_5; DROP TABLE wtd_comp_prep;

In [0]:
-- % LY comp calculations
DROP TABLE IF EXISTS comp_wtd_report CASCADE;
CREATE TABLE comp_wtd_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM comp_wtd AS f LEFT JOIN comp_wtd AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

DROP TABLE comp_wtd;

In [0]:
-- calculating month to date metrics
CALL sp_dr_to_date('month', 'mtd_part1', 'report_part_3');

-- calculating rest of the month to date metrics
CALL sp_dr_num_den('mtd_part2', 'mtd_part1');

In [0]:
CALL sp_dr_agg_by('mtd_part3_base', 'mtd_part1', 'total', 'month', FALSE);

CALL sp_dr_num_den('mtd_part3', 'mtd_part3_base');

DROP TABLE mtd_part3_base;

In [0]:
CALL sp_dr_agg_by('mtd_part4_base', 'mtd_part1', 'in-stores', 'month', FALSE);

CALL sp_dr_num_den('mtd_part4', 'mtd_part4_base');

DROP TABLE mtd_part4_base;

In [0]:
CALL sp_dr_agg_by('mtd_part5_base', 'mtd_part1', 'online', 'month', FALSE);

CALL sp_dr_num_den('mtd_part5', 'mtd_part5_base');

DROP TABLE mtd_part5_base;

In [0]:
CALL sp_dr_agg_by('mtd_part6_base', 'mtd_part1', 'omni', 'month', FALSE);

CALL sp_dr_num_den('mtd_part6', 'mtd_part6_base');

DROP TABLE mtd_part6_base;

In [0]:
CALL sp_dr_agg_by('mtd_part7_base', 'mtd_part1', 'region', 'month', FALSE);

CALL sp_dr_num_den('mtd_part7', 'mtd_part7_base');

DROP TABLE mtd_part7_base;

In [0]:
-- creating the mtd report table by unioning all the mtd parts
DROP TABLE IF EXISTS mtd_report CASCADE;
CREATE TABLE mtd_report AS
SELECT * FROM mtd_part2 UNION
SELECT * FROM mtd_part3 UNION
SELECT * FROM mtd_part4 UNION
SELECT * FROM mtd_part5 UNION
SELECT * FROM mtd_part6 UNION
SELECT * FROM mtd_part7;

DROP TABLE mtd_part2; DROP TABLE mtd_part3;
DROP TABLE mtd_part4; DROP TABLE mtd_part5;
DROP TABLE mtd_part6; DROP TABLE mtd_part7;

In [0]:
-- % LY calculations
DROP TABLE IF EXISTS ly_mtd_report CASCADE;
CREATE TABLE ly_mtd_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM mtd_report AS f LEFT JOIN mtd_report AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

In [0]:
CREATE TEMP TABLE mtd_comp_prep AS
SELECT NVL(grand_open_date, '2020-01-01') :: date AS store_open_date, r.*
FROM mtd_part1 AS r LEFT JOIN store_lookup AS l
ON r.profit_center :: int = l.pft_ctr AND r.app_key = l.app_key;

In [0]:
CALL sp_dr_agg_by('mtd_comp_1_base', 'mtd_comp_prep', 'total', 'month', TRUE);

CALL sp_dr_num_den('mtd_comp_1', 'mtd_comp_1_base');

DROP TABLE mtd_comp_1_base;

In [0]:
CALL sp_dr_agg_by('mtd_comp_2_base', 'mtd_comp_prep', 'in-stores', 'month', TRUE);

CALL sp_dr_num_den('mtd_comp_2', 'mtd_comp_2_base');

DROP TABLE mtd_comp_2_base;

In [0]:
CALL sp_dr_agg_by('mtd_comp_3_base', 'mtd_comp_prep', 'online', 'month', TRUE);

CALL sp_dr_num_den('mtd_comp_3', 'mtd_comp_3_base');

DROP TABLE mtd_comp_3_base;

In [0]:
CALL sp_dr_agg_by('mtd_comp_4_base', 'mtd_comp_prep', 'omni', 'month', TRUE);

CALL sp_dr_num_den('mtd_comp_4', 'mtd_comp_4_base');

DROP TABLE mtd_comp_4_base;

In [0]:
CALL sp_dr_agg_by('mtd_comp_5_base', 'mtd_comp_prep', 'region', 'month', TRUE);

CALL sp_dr_num_den('mtd_comp_5', 'mtd_comp_5_base');

DROP TABLE mtd_comp_5_base;

In [0]:
CREATE TEMP TABLE comp_mtd AS
SELECT * FROM mtd_comp_1 UNION
SELECT * FROM mtd_comp_2 UNION
SELECT * FROM mtd_comp_3 UNION
SELECT * FROM mtd_comp_4 UNION
SELECT * FROM mtd_comp_5;

DROP TABLE mtd_comp_1; DROP TABLE mtd_comp_2;
DROP TABLE mtd_comp_3; DROP TABLE mtd_comp_4;
DROP TABLE mtd_comp_5; DROP TABLE mtd_comp_prep;

In [0]:
-- % LY comp calculations
DROP TABLE IF EXISTS comp_mtd_report CASCADE;
CREATE TABLE comp_mtd_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM comp_mtd AS f LEFT JOIN comp_mtd AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

DROP TABLE comp_mtd;

In [0]:
-- calculating year to date metrics
CALL sp_dr_to_date('year', 'ytd_part1', 'report_part_3');

-- calculating rest of the year to date metrics
CALL sp_dr_num_den('ytd_part2', 'ytd_part1');

In [0]:
CALL sp_dr_agg_by('ytd_part3_base', 'ytd_part1', 'total', 'year', FALSE);

CALL sp_dr_num_den('ytd_part3', 'ytd_part3_base');

DROP TABLE ytd_part3_base;

In [0]:
CALL sp_dr_agg_by('ytd_part4_base', 'ytd_part1', 'in-stores', 'year', FALSE);

CALL sp_dr_num_den('ytd_part4', 'ytd_part4_base');

DROP TABLE ytd_part4_base;

In [0]:
CALL sp_dr_agg_by('ytd_part5_base', 'ytd_part1', 'online', 'year', FALSE);

CALL sp_dr_num_den('ytd_part5', 'ytd_part5_base');

DROP TABLE ytd_part5_base;

In [0]:
CALL sp_dr_agg_by('ytd_part6_base', 'ytd_part1', 'omni', 'year', FALSE);

CALL sp_dr_num_den('ytd_part6', 'ytd_part6_base');

DROP TABLE ytd_part6_base;

In [0]:
CALL sp_dr_agg_by('ytd_part7_base', 'ytd_part1', 'region', 'year', FALSE);

CALL sp_dr_num_den('ytd_part7', 'ytd_part7_base');

DROP TABLE ytd_part7_base;

In [0]:
-- creating the mtd report table by unioning all the mtd parts
DROP TABLE IF EXISTS ytd_report CASCADE;
CREATE TABLE ytd_report AS
SELECT * FROM ytd_part2 UNION
SELECT * FROM ytd_part3 UNION
SELECT * FROM ytd_part4 UNION
SELECT * FROM ytd_part5 UNION
SELECT * FROM ytd_part6 UNION
SELECT * FROM ytd_part7;

DROP TABLE ytd_part2; DROP TABLE ytd_part3;
DROP TABLE ytd_part4; DROP TABLE ytd_part5;
DROP TABLE ytd_part6; DROP TABLE ytd_part7;

In [0]:
-- % LY calculations
DROP TABLE IF EXISTS ly_ytd_report CASCADE;
CREATE TABLE ly_ytd_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM ytd_report AS f LEFT JOIN ytd_report AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

In [0]:
CREATE TEMP TABLE ytd_comp_prep AS
SELECT NVL(grand_open_date, '2020-01-01') :: date AS store_open_date, r.*
FROM ytd_part1 AS r LEFT JOIN store_lookup AS l
ON r.profit_center :: int = l.pft_ctr AND r.app_key = l.app_key;

In [0]:
CALL sp_dr_agg_by('ytd_comp_1_base', 'ytd_comp_prep', 'total', 'year', TRUE);

CALL sp_dr_num_den('ytd_comp_1', 'ytd_comp_1_base');

DROP TABLE ytd_comp_1_base;

In [0]:
CALL sp_dr_agg_by('ytd_comp_2_base', 'ytd_comp_prep', 'in-stores', 'year', TRUE);

CALL sp_dr_num_den('ytd_comp_2', 'ytd_comp_2_base');

DROP TABLE ytd_comp_2_base;

In [0]:
CALL sp_dr_agg_by('ytd_comp_3_base', 'ytd_comp_prep', 'online', 'year', TRUE);

CALL sp_dr_num_den('ytd_comp_3', 'ytd_comp_3_base');

DROP TABLE ytd_comp_3_base;

In [0]:
CALL sp_dr_agg_by('ytd_comp_4_base', 'ytd_comp_prep', 'omni', 'year', TRUE);

CALL sp_dr_num_den('ytd_comp_4', 'ytd_comp_4_base');

DROP TABLE ytd_comp_4_base;

In [0]:
CALL sp_dr_agg_by('ytd_comp_5_base', 'ytd_comp_prep', 'region', 'year', TRUE);

CALL sp_dr_num_den('ytd_comp_5', 'ytd_comp_5_base');

DROP TABLE ytd_comp_5_base;

In [0]:
CREATE TEMP TABLE comp_ytd AS
SELECT * FROM ytd_comp_1 UNION
SELECT * FROM ytd_comp_2 UNION
SELECT * FROM ytd_comp_3 UNION
SELECT * FROM ytd_comp_4 UNION
SELECT * FROM ytd_comp_5;

DROP TABLE ytd_comp_1; DROP TABLE ytd_comp_2;
DROP TABLE ytd_comp_3; DROP TABLE ytd_comp_4;
DROP TABLE ytd_comp_5; DROP TABLE ytd_comp_prep;

In [0]:
-- % LY comp calculations
DROP TABLE IF EXISTS comp_ytd_report CASCADE;
CREATE TABLE comp_ytd_report AS
SELECT f.report_date, f.app_key, f.profit_center, f.custom_key, f.filter_level,
    f.segment, f.store_name, f.region,

    TRUNC(ROUND((f.traffic - s.traffic) * 100
    / NULLIF(s.traffic, 0), 2), 2) AS traffic,
    
    TRUNC(ROUND((f.total_sales_amt - s.total_sales_amt) * 100
    / NULLIF(s.total_sales_amt, 0), 2), 2) AS total_sales_amt,
    
    TRUNC(ROUND((f.stores_sales_amt - s.stores_sales_amt) * 100
    / NULLIF(s.stores_sales_amt, 0), 2), 2) AS stores_sales_amt,
    
    TRUNC(ROUND((f.online_sales_amt - s.online_sales_amt) * 100
    / NULLIF(s.online_sales_amt, 0), 2), 2) AS online_sales_amt,
    
    TRUNC(ROUND((f.omni_sales_amt - s.omni_sales_amt) * 100
    / NULLIF(s.omni_sales_amt, 0), 2), 2) AS omni_sales_amt,
    
    TRUNC(ROUND((f.cancelled_sales_amt - s.cancelled_sales_amt) * 100
    / NULLIF(s.cancelled_sales_amt, 0), 2), 2) AS cancelled_sales_amt,
    
    TRUNC(ROUND((f.gross_margin_pct - s.gross_margin_pct) * 100
    / NULLIF(s.gross_margin_pct, 0), 2), 2) AS gross_margin_pct,
    
    TRUNC(ROUND((f.stores_gross_margin_amt - s.stores_gross_margin_amt) * 100
    / NULLIF(s.stores_gross_margin_amt, 0), 2), 2) AS stores_gross_margin_amt,
    
    TRUNC(ROUND((f.stores_gross_margin_pct - s.stores_gross_margin_pct) * 100
    / NULLIF(s.stores_gross_margin_pct, 0), 2), 2) AS stores_gross_margin_pct,
    
    TRUNC(ROUND((f.initial_markup_pct - s.initial_markup_pct) * 100
    / NULLIF(s.initial_markup_pct, 0), 2), 2) AS initial_markup_pct,

    TRUNC(ROUND((f.item_discount_pct - s.item_discount_pct) * 100
    / NULLIF(s.item_discount_pct, 0), 2), 2) AS item_discount_pct,
    
    TRUNC(ROUND((f.orders_with_discount_pct - s.orders_with_discount_pct) * 100
    / NULLIF(s.orders_with_discount_pct, 0), 2), 2) AS orders_with_discount_pct,
    
    TRUNC(ROUND((f.effective_margin_amt - s.effective_margin_amt) * 100
    / NULLIF(s.effective_margin_amt, 0), 2), 2) AS effective_margin_amt,
    
    TRUNC(ROUND((f.effective_margin_pct - s.effective_margin_pct) * 100
    / NULLIF(s.effective_margin_pct, 0), 2), 2) AS effective_margin_pct,
    
    TRUNC(ROUND((f.financing_cost_pct - s.financing_cost_pct) * 100
    / NULLIF(s.financing_cost_pct, 0), 2), 2) AS financing_cost_pct,
    
    TRUNC(ROUND((f.avg_ticket_amt - s.avg_ticket_amt) * 100
    / NULLIF(s.avg_ticket_amt, 0), 2), 2) AS avg_ticket_amt,
    
    TRUNC(ROUND((f.items_per_ticket - s.items_per_ticket) * 100
    / NULLIF(s.items_per_ticket, 0), 2), 2) AS items_per_ticket,
    
    TRUNC(ROUND((f.avg_sales_price - s.avg_sales_price) * 100
    / NULLIF(s.avg_sales_price, 0), 2), 2) AS avg_sales_price,
    
    TRUNC(ROUND((f.close_rate_pct - s.close_rate_pct) * 100
    / NULLIF(s.close_rate_pct, 0), 2), 2) AS close_rate_pct,
    
    TRUNC(ROUND((f.carts_built_pct - s.carts_built_pct) * 100
    / NULLIF(s.carts_built_pct, 0), 2), 2) AS carts_built_pct,
    
    TRUNC(ROUND((f.sales_per_guest - s.sales_per_guest) * 100
    / NULLIF(s.sales_per_guest, 0), 2), 2) AS sales_per_guest,
    
    TRUNC(ROUND((f.total_financed_amt - s.total_financed_amt) * 100
    / NULLIF(s.total_financed_amt, 0), 2), 2) AS total_financed_amt,
    
    TRUNC(ROUND((f.financed_sales_pct - s.financed_sales_pct) * 100
    / NULLIF(s.financed_sales_pct, 0), 2), 2) AS financed_sales_pct,
    
    TRUNC(ROUND((f.avg_financed_ticket_amt - s.avg_financed_ticket_amt) * 100
    / NULLIF(s.avg_financed_ticket_amt, 0), 2), 2) AS avg_financed_ticket_amt,
    
    TRUNC(ROUND((f.finance_application_pct - s.finance_application_pct) * 100
    / NULLIF(s.finance_application_pct, 0), 2), 2) AS finance_application_pct,
    
    TRUNC(ROUND((f.sales_per_hour - s.sales_per_hour) * 100
    / NULLIF(s.sales_per_hour, 0), 2), 2) AS sales_per_hour,
    
    TRUNC(ROUND((f.bedding_sales_amt - s.bedding_sales_amt) * 100
    / NULLIF(s.bedding_sales_amt, 0), 2), 2) AS bedding_sales_amt,
    
    TRUNC(ROUND((f.bedding_spg - s.bedding_spg) * 100
    / NULLIF(s.bedding_spg, 0), 2), 2) AS bedding_spg,
    
    TRUNC(ROUND((f.bedding_sph - s.bedding_sph) * 100
    / NULLIF(s.bedding_sph, 0), 2), 2) AS bedding_sph,
    
    TRUNC(ROUND((f.bedding_sales_pct - s.bedding_sales_pct) * 100
    / NULLIF(s.bedding_sales_pct, 0), 2), 2) AS bedding_sales_pct,
    
    TRUNC(ROUND((f.bedding_gross_margin_pct - s.bedding_gross_margin_pct) * 100
    / NULLIF(s.bedding_gross_margin_pct, 0), 2), 2) AS bedding_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_financing_cost - s.bedding_financing_cost) * 100
    / NULLIF(s.bedding_financing_cost, 0), 2), 2) AS bedding_financing_cost,
    
    TRUNC(ROUND((f.furniture_sales_amt - s.furniture_sales_amt) * 100
    / NULLIF(s.furniture_sales_amt, 0), 2), 2) AS furniture_sales_amt,
    
    TRUNC(ROUND((f.furniture_spg - s.furniture_spg) * 100
    / NULLIF(s.furniture_spg, 0), 2), 2) AS furniture_spg,
    
    TRUNC(ROUND((f.furniture_sph - s.furniture_sph) * 100
    / NULLIF(s.furniture_sph, 0), 2), 2) AS furniture_sph,
    
    TRUNC(ROUND((f.furniture_sales_pct - s.furniture_sales_pct) * 100
    / NULLIF(s.furniture_sales_pct, 0), 2), 2) AS furniture_sales_pct,
    
    TRUNC(ROUND((f.furniture_gross_margin_pct - s.furniture_gross_margin_pct) * 100
    / NULLIF(s.furniture_gross_margin_pct, 0), 2), 2) AS furniture_gross_margin_pct,
    
    TRUNC(ROUND((f.furniture_financing_cost - s.furniture_financing_cost) * 100
    / NULLIF(s.furniture_financing_cost, 0), 2), 2) AS furniture_financing_cost,
    
    TRUNC(ROUND((f.protection_sales_amt - s.protection_sales_amt) * 100
    / NULLIF(s.protection_sales_amt, 0), 2), 2) AS protection_sales_amt,
    
    TRUNC(ROUND((f.protection_spg - s.protection_spg) * 100
    / NULLIF(s.protection_spg, 0), 2), 2) AS protection_spg,
    
    TRUNC(ROUND((f.protection_sph - s.protection_sph) * 100
    / NULLIF(s.protection_sph, 0), 2), 2) AS protection_sph,
    
    TRUNC(ROUND((f.protection_sales_pct - s.protection_sales_pct) * 100
    / NULLIF(s.protection_sales_pct, 0), 2), 2) AS protection_sales_pct,
    
    TRUNC(ROUND((f.delivery_sales_amt - s.delivery_sales_amt) * 100
    / NULLIF(s.delivery_sales_amt, 0), 2), 2) AS delivery_sales_amt,
    
    TRUNC(ROUND((f.delivery_spg - s.delivery_spg) * 100
    / NULLIF(s.delivery_spg, 0), 2), 2) AS delivery_spg,
    
    TRUNC(ROUND((f.delivery_sales_pct - s.delivery_sales_pct) * 100
    / NULLIF(s.delivery_sales_pct, 0), 2), 2) AS delivery_sales_pct,
    
    TRUNC(ROUND((f.sales_delivered - s.sales_delivered) * 100
    / NULLIF(s.sales_delivered, 0), 2), 2) AS sales_delivered,
    
    TRUNC(ROUND((f.avg_days_to_deliver - s.avg_days_to_deliver) * 100
    / NULLIF(s.avg_days_to_deliver, 0), 2), 2) AS avg_days_to_deliver,
    
    TRUNC(ROUND((f.delivered_gross_margin_pct - s.delivered_gross_margin_pct) * 100
    / NULLIF(s.delivered_gross_margin_pct, 0), 2), 2) AS delivered_gross_margin_pct,
    
    TRUNC(ROUND((f.bedding_sales_delivered - s.bedding_sales_delivered) * 100
    / NULLIF(s.bedding_sales_delivered, 0), 2), 2) AS bedding_sales_delivered

FROM comp_ytd AS f LEFT JOIN comp_ytd AS s
ON f.ly_date = s.report_date AND f.segment = s.segment;

DROP TABLE comp_ytd;

In [0]:
CREATE VIEW users_views.daily_report_vw AS
SELECT * FROM analytics.daily_report;

CREATE VIEW users_views.wtd_report_vw AS
SELECT * FROM analytics.wtd_report;

CREATE VIEW users_views.mtd_report_vw AS
SELECT * FROM analytics.mtd_report;

CREATE VIEW users_views.ytd_report_vw AS
SELECT * FROM analytics.ytd_report;

In [0]:
CREATE VIEW users_views.ly_daily_report_vw AS
SELECT * FROM analytics.ly_daily_report;

CREATE VIEW users_views.ly_wtd_report_vw AS
SELECT * FROM analytics.ly_wtd_report;

CREATE VIEW users_views.ly_mtd_report_vw AS
SELECT * FROM analytics.ly_mtd_report;

CREATE VIEW users_views.ly_ytd_report_vw AS
SELECT * FROM analytics.ly_ytd_report;

In [0]:
CREATE VIEW users_views.comp_daily_report_vw AS
SELECT * FROM analytics.comp_daily_report;

CREATE VIEW users_views.comp_wtd_report_vw AS
SELECT * FROM analytics.comp_wtd_report;

CREATE VIEW users_views.comp_mtd_report_vw AS
SELECT * FROM analytics.comp_mtd_report;

CREATE VIEW users_views.comp_ytd_report_vw AS
SELECT * FROM analytics.comp_ytd_report;